In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1633172146223_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = spark.read.csv('s3://project021021/olist_public_dataset.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_sellers_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string

In [4]:
df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+------------+--------------------+-------------------+---------------+-----------------+------------------------+--------------------+-----------------------------+-----------------------------+-------------+--------------+------------------------+---------------------+-------------------+--------------------------+------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|_c0| id|order_status|order_products_value|order_freight_value|order_items_qty|order_sellers_qty|order_purchase_timestamp|    order_aproved_at|order_estimated_delivery_date|order_delivered_customer_date|customer_city|customer_state|customer_zip_code_prefix|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---+---+------------+--------------------+-------------------+---------------+-----------------+-------

In [5]:
from pyspark.sql.functions import *

df2 = df.withColumn("ts", to_timestamp(col("order_purchase_timestamp"))) \
    .withColumn("datetype", to_date(col("order_purchase_timestamp"),"yyyy-MM-dd"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df3.write.format('csv').option('header','true').save('s3://project021021/out/olist_classified_public_dataset_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df3.select('week_of_month', 'day_of_week').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------+
|week_of_month|day_of_week|
+-------------+-----------+
|            2|          4|
|            2|          2|
|            4|          5|
|            3|          5|
|            1|          7|
|            2|          6|
|            1|          7|
|            2|          5|
|            2|          4|
|            3|          4|
+-------------+-----------+
only showing top 10 rows

In [11]:
df4 = df3.groupBy('order_status', 'week_of_month', 'day_of_week', 'customer_city').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df4.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-----------+--------------------+-----+
|order_status|week_of_month|day_of_week|       customer_city|count|
+------------+-------------+-----------+--------------------+-----+
|   delivered|            3|          1|       Volta Redonda|    5|
|   delivered|            1|          6|            Anapolis|    5|
|   delivered|            3|          5|Santo Antonio do ...|    1|
|   delivered|            2|          5|        Quirinopolis|    1|
|   delivered|            2|          7|           Cruz Alta|    1|
|   delivered|            5|          6|   Sao Mateus do Sul|    3|
|   delivered|            2|          2|     Pindamonhangaba|    1|
|   delivered|            3|          1|             Ibirite|    2|
|   delivered|            6|          1|           Cravinhos|    1|
|  processing|            1|          6|               Aruja|    1|
|   delivered|            4|          3|            BRASILIA|   64|
|   delivered|            3|          1|        

In [13]:
df4.write.format('csv').option('header','true').save('s3://project021021/out/olist_classified_public_dataset_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
average_review_score = df3.select(avg('review_score')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
approved_order_count = df3.filter(col('order_aproved_at').isNotNull() ).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
total_order_value = df3.agg(sum('order_products_value')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(f'Average order approval = {total_order_value/approved_order_count}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Average order approval = 128.43788861993937

In [18]:
average_frieght_value = df3.select(avg('order_freight_value')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df3.select(avg('review_score').alias('average_review_score'), 
           avg('order_freight_value').alias('average_frieght_value'), 
           avg('order_products_value').alias('Average_order_approval')) \
.first()[0:3]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(4.05045, 21.741279200000008, 128.4147697999878)

In [20]:
df5 = df3.withColumn('order_deliver_timestamp', to_timestamp(col('order_delivered_customer_date'))) \
        .withColumn('order_purchase_timestamp_conv', to_timestamp(col('order_purchase_timestamp')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df5.select('order_deliver_timestamp', 'order_purchase_timestamp_conv').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------------------+
|order_deliver_timestamp|order_purchase_timestamp_conv|
+-----------------------+-----------------------------+
|   2017-11-23 21:21:...|          2017-11-08 13:19:33|
|   2017-08-17 19:49:...|          2017-08-07 11:23:16|
|   2018-04-15 17:56:...|          2018-03-22 02:29:42|
|   2017-10-03 13:13:...|          2017-09-14 21:51:28|
|   2018-03-06 16:41:...|          2018-03-03 20:50:21|
|   2017-04-11 05:42:...|          2017-04-07 12:19:51|
|   2017-07-19 20:19:...|          2017-07-01 22:38:39|
|   2018-08-21 14:59:...|          2018-08-09 11:22:34|
|   2018-07-31 15:27:...|          2018-07-11 08:17:08|
|   2017-07-07 20:32:...|          2017-06-14 18:31:54|
+-----------------------+-----------------------------+
only showing top 10 rows

In [22]:
df6 = df5.withColumn('deliver_time', 
               col('order_deliver_timestamp').cast('long')- 
               col('order_purchase_timestamp_conv').cast('long'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df6.select('deliver_time').first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(deliver_time=1324912)

In [24]:
average_delivery_time = df6.select(avg('deliver_time')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
total_frieght_charges = df.select(sum('order_freight_value')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df.groupBy('customer_city').agg(sum('order_freight_value')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+
|       customer_city|sum(order_freight_value)|
+--------------------+------------------------+
|           Carrancas|      48.260000000000005|
|            Machados|                  127.52|
|    Francisco Morato|                 1279.74|
|             Aracati|       760.6800000000001|
|           Agua Doce|       76.08000000000001|
|Brasilandia de Minas|                   92.24|
|           Americana|      3382.6099999999997|
|            Macatuba|      328.78000000000003|
|           Nova Lima|      2043.5100000000007|
|               Nipoa|                   28.65|
|           Chorrocho|      157.01000000000002|
|          Pedra Azul|                  201.24|
|             Parapua|                    55.4|
|        Francisco Sa|                  121.13|
|        Virginopolis|      237.62000000000003|
|            Antonina|                  246.24|
|         Turvolandia|                   80.02|
|             Fartura|                  

In [33]:
df6.write.csv('s3://project021021/out/olist_classified_public_dataset_final.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df6.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_sellers_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string